# Importing packages and datasets.

In [1]:
import pandas
import numpy
import pickle

import altair

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error


In [2]:
dataset_file_path = "../data/processed/"


In [3]:
dataFrame = pandas.read_csv(dataset_file_path + "cleaned.csv")
print(dataFrame.shape)


(2895, 30)


In [4]:
normalisedDataFrame = pandas.read_csv(dataset_file_path + "normalised.csv")
print(normalisedDataFrame.shape)


(2895, 30)


In [5]:
print(dataFrame.columns)


Index(['avgAnnCount', 'avgDeathsPerYear', 'TARGET_deathRate', 'incidenceRate',
       'medIncome', 'popEst2015', 'povertyPercent', 'studyPerCap', 'MedianAge',
       'MedianAgeMale', 'MedianAgeFemale', 'AvgHouseholdSize',
       'PercentMarried', 'PctNoHS18_24', 'PctHS18_24', 'PctBachDeg18_24',
       'PctHS25_Over', 'PctBachDeg25_Over', 'PctEmployed16_Over',
       'PctUnemployed16_Over', 'PctPrivateCoverage', 'PctEmpPrivCoverage',
       'PctPublicCoverage', 'PctPublicCoverageAlone', 'PctWhite', 'PctBlack',
       'PctAsian', 'PctOtherRace', 'PctMarriedHouseholds', 'BirthRate'],
      dtype='object')


In [6]:
possibly_related_fields = [
                           "medIncome", "povertyPercent", "PctUnemployed16_Over", "PctEmployed16_Over",
                           "PctHS25_Over", "PctBachDeg25_Over",
                           "PctPrivateCoverage", "PctEmpPrivCoverage", "PctPublicCoverage", "PctPublicCoverageAlone",
                           "PctMarriedHouseholds", "PercentMarried"
                           ]


In [7]:
X = dataFrame[possibly_related_fields + ["TARGET_deathRate"]].copy()
y = X.pop("TARGET_deathRate")

train, test = train_test_split(dataFrame, test_size=0.1727, random_state=42)
df_train, df_test = train_test_split(X, test_size=0.1727, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.1727, random_state=42)

print(df_train.shape, "\t", df_test.shape)
print(y_train.shape, "\t", y_test.shape)


(2395, 12) 	 (500, 12)
(2395,) 	 (500,)


In [8]:
X = normalisedDataFrame[possibly_related_fields + ["TARGET_deathRate"]].copy()
y = X.pop("TARGET_deathRate")

train, test = train_test_split(dataFrame, test_size=0.1727, random_state=42)
df_train, df_test = train_test_split(X, test_size=0.1727, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.1727, random_state=42)

print(df_train.shape, "\t", df_test.shape)
print(y_train.shape, "\t", y_test.shape)


(2395, 12) 	 (500, 12)
(2395,) 	 (500,)


# Baseline Performance.

In [9]:
y_mean = y.mean()

y_base = numpy.full((y_test.shape[0], ), y_mean)
(y_base - y_test).sum()

mean_squared_error(y_test, y_base, squared=True)


712.9935696334397

# Training.

In [10]:
regression = LinearRegression()

regression.fit(df_train, y_train)


LinearRegression()

In [11]:
model_file_path = "../models/"
model_file_name = "linear_multivariate.pickle"


In [12]:
pickle.dump(regression, open(model_file_path + model_file_name, "wb"))


In [13]:
regression = pickle.load(open(model_file_path + model_file_name, "rb"))


# Testing.

In [14]:
def results(test_type):

	mse = 0.0
	prediction_line = altair.Chart()
	actual_line = altair.Chart()
	line_chart = altair.Chart()

	if test_type == "test":
		y_preds = regression.predict(df_test)
		prediction_line = altair.Chart(pandas.DataFrame({'x': y_test, 'y': y_preds})).mark_line(color='blue').encode(
      		x='x',
      		y='y'
    	)
		actual_line = altair.Chart(pandas.DataFrame({'x': y_test, 'y': y_test})).mark_line(color='orange').encode(
      		x='x',
      		y='y'
    	)
		mse = mean_squared_error(y_test, y_preds, squared=True)

	elif test_type == "train":
		y_preds = regression.predict(df_train)
		prediction_line = altair.Chart(pandas.DataFrame({'x': y_train, 'y': y_preds})).mark_line(color='blue').encode(
      		x='x',
      		y='y'
    	)
		actual_line = altair.Chart(pandas.DataFrame({'x': y_train, 'y': y_train})).mark_line(color='orange').encode(
      		x='x',
      		y='y'
    	)
		mse = mean_squared_error(y_train, y_preds, squared=True)

	return (actual_line, prediction_line, mse)


In [15]:
actual_line, prediction_line, mse = results("train")
print(mse)


489.20877939748414


In [16]:
actual_line, prediction_line, mse = results("test")
print(mse)


438.2299382077526
